### Task 1

This application detects and tracks moving vehicles from a given camera recording. The application uses background subtraction by utilizing the BackgroundSubtractorMOG2 method from OpenCV. Background subtraction is a simple, yet effective technique to extract images from video. 

In [2]:
!pip3 install opencv-python
!pip3 install numpy

In [1]:
import os.path
import cv2
import numpy as np
import time
import sys


def VehicleTracker(path):
    ''' Function that tracks vehicles in recording'''
    if os.path.isfile(path):
        
        recording = cv2.VideoCapture(path)
        
        # Initialize the background object using background subtractor
        BS_MOG2 = cv2.createBackgroundSubtractorMOG2(detectShadows = True)
        
        while recording.isOpened():
    
            # Read a new frame.
            frameAvailable, frame = recording.read()

            # Check if frame is available, i.e the file is a video.
            if not frameAvailable:
                break
        
            # Apply the background object on the frame. 
            segmentedMask = BS_MOG2.apply(frame)
    
            #Thresholding to remove shadows.
            _, threshold = cv2.threshold(segmentedMask, 250, 255, cv2.THRESH_BINARY)
    
            # Apply morphological operations to get a good mask
            foregroundMask = cv2.erode(threshold, None, iterations = 1)
            fgMask = cv2.dilate(foregroundMask, None, iterations = 2)
    
            # Detect contours in the frame
            contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
            #Copy of the frame to draw bounding boxes.
            frameCopy = frame.copy()
    
            # loop over each contour in the frame.
            for contour in contours:
        
                # Threshold of 3500, which is roughly the contour area of a car
                if cv2.contourArea(contour) < 3500:
                    continue;
            
                # Retrieve the bounding box coordinates from the contour.
                x, y, width, height = cv2.boundingRect(contour)

                # Draw a bounding box around the car.
                cv2.rectangle(frameCopy, (x , y), (x + width, y + height),(0, 255, 0), 2)
            
    
            # Extract the foreground from the frame using the segmented mask.
            foreground = cv2.bitwise_and(frame, frame, mask=fgMask)
        
            # Stack the original frame, foreground, and detection frame. 
            stacked = np.hstack((frame, foreground, frameCopy))
    
            #Wait until a key is pressed
            key = cv2.waitKey(1) & 0xff
    
            #press x to exit tracker
            if key == ord('x'):
                recording.release()
                cv2.destroyAllWindows()
                print("END OF PROCESSING")
                break
        
    
            # Display the stacked image with an appropriate title.
            cv2.imshow('ORIGINAL FRAME | EXTRACTED FOREGROUND  | DETECTED VEHICLES', cv2.resize(stacked, None, fx=1.0, fy=1.0))

## How to start and stop processing

To start detecting cars, enter the recording number (1 or 2). A window will open showing the original video and the annotated video. To exist detection, press x. To exit the application, you must press the "q" key. 

In [2]:
# video path files
path1 = "Traffic_Laramie_1.mp4"
path2 = "Traffic_Laramie_2.mp4"

while True:
    val = input("Enter 1 for first recording, 2 for second recording, q to quit: ")

    if val == "1":
        VehicleTracker(path1)
    elif val == "2":
        VehicleTracker(path2)
    elif val == "q":
        break
    else:
        print("Invalid file path!")

#Quit the application
exit()

Enter 1 for first recording, 2 for second recording, q to quit: 1
END OF PROCESSING
Enter 1 for first recording, 2 for second recording, q to quit: q


### Task 2

In the second task, we use frame differencing to track vehicles going from the city center to the downtown area over a time interval. This is done by

In [1]:
import os.path
import cv2
import numpy as np
import time
import math

def VehicleCounter(path):
    ''' Function that tracks and counts vehicles in a recording'''
    
    recording = cv2.VideoCapture(path)
    
    if not os.path.isfile(path):
        raise ValueError("Video File is not valid")
        
    #get video
    recording = cv2.VideoCapture(path)
    fps = recording.get(cv2.CAP_PROP_FPS) 
        
    #attempt to read frames
    ready, frame_prev = recording.read()
        
    if not ready:
        raise ValueError("File is not video and has no frames to read!")
        
    #vehicle count    
    vehicle_count = 0
    while recording.isOpened():
    
        # Read a new frame.
        available, frame_next = recording.read()

        # Check if frame is available, i.e the file is a video.
        if not available:
            break
        
        #copy existing frame
        frame = frame_next.copy()

        # Apply frame differencing 
        fgMask = cv2.absdiff(frame_prev,frame_next)

        #convert to gry
        fgMask = cv2.cvtColor(fgMask, cv2.COLOR_BGR2GRAY)

        #Apply threshold
        _, threshold = cv2.threshold(fgMask, 5, 255, cv2.THRESH_BINARY)
        

        #Consecutive frame, frame count, and duration
        frame_prev = frame_next
        
        minutes = math.ceil(recording.get(cv2.CAP_PROP_POS_MSEC)/1000.00/60.0)
        
        #Set minutes
        if minutes == 0:
            frame_number = recording.get(cv2.CAP_PROP_FRAME_COUNT)
            fps = int(recording.get(cv2.CAP_PROP_FPS))
            minutes = int(frame_number / fps)
        

        #draw reference line
        cv2.line(frame, (340, 460), (373, 350), (0,0,255), 1) #red line
        cv2.line(frame, (345, 460), (378, 350), (0,255,0), 2) #green line
        cv2.line(frame, (350, 460), (383, 350), (255,0,0), 1) #blue line
        
        # Detect contours in the frame
        contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # loop over each contour in the frame.
        for contour in contours:
            
            #Skip noise contours
            if cv2.contourArea(contour) < 3250:
                continue

            # Retrieve the bounding box coordinates from the contour.
            x, y, width, height = cv2.boundingRect(contour)
            

            # Draw a bounding box around the car.
            cv2.rectangle(frame, (x , y), (x + width, y + height),(0, 255, 0), 2)
            
            #Get mid points of rectangle
            xMid = int((x + (x+width))/2)
            yMid = int((y + (y+height))/2)

            cv2.circle(frame,(xMid,yMid),5, (0,0,255),2)
            
            if xMid >= 337 and xMid <= 342 and yMid < 460:
                vehicle_count += 1

            
  
          
        #display number of cars heading to city center    
        cv2.putText(frame, "Vehicles CC Bound: {}".format(vehicle_count), (550,50), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))
        cv2.putText(frame, "Cars/min: {}".format(int(vehicle_count/minutes)), (550,100), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255))  
            
        # Display the stacked image with an appropriate title.
        cv2.imshow('Foreground', cv2.resize(fgMask, None, fx=1.0, fy=1.0))
        cv2.imshow('Car Detection', cv2.resize(frame, None, fx=1.0, fy=1.0))

        #Wait until a key is pressed
        key = cv2.waitKey(1) & 0xff

        #press x to exit tracker
        if key == ord('x'):
            recording.release()
            cv2.destroyAllWindows()
            print("END OF PROCESSING")
            break

In [ ]:
# video path files
path1 = "Traffic_Laramie_1.mp4"
path2 = "Traffic_Laramie_2.mp4"

while True:
    val = input("Enter 1 for first recording, 2 for second recording, q to quit: ")

    if val == "1":
        VehicleCounter(path1)
    elif val == "2":
        VehicleCounter(path2)
    elif val == "q":
        break
    else:
        print("Invalid file path!")

#Quit the application
exit()

Enter 1 for first recording, 2 for second recording, q to quit: 2
